In [1]:
import matplotlib.pyplot as plt
!pip install map-boxes

You should consider upgrading via the '/home/ruslan/miniconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
from pathlib import Path
import random

Couldn't import fast version of function compute_overlap, will use slow one. Check cython intallation


In [3]:
!git clone https://github.com/WongKinYiu/yolov7
!pip install -r requirements.txt

fatal: destination path 'yolov7' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
You should consider upgrading via the '/home/ruslan/miniconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
%cd yolov7

/home/ruslan/Downloads/Telegram Desktop/Татарстан/yolov7


In [ ]:
# Run evaluation
# !python detect.py --weights ../best3.pt --conf 0.4 --source data_for_yolo/data/images/test --save-txt --save-conf
!python detect.py --weights ../final.pt --conf 0.001 --source ../test_dataset_test/test/images --save-txt --save-conf --nosave
# !python detect.py --weights ../best8.pt --conf 0.1 --source data_for_yolo/data/images/test --save-txt --save-conf --nosave

Namespace(weights=['../final.pt'], source='../test_dataset_test/test/images', img_size=640, conf_thres=0.001, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=True, nosave=True, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 2022-7-28 torch 1.10.1 CPU

Fusing layers... 
/home/ruslan/miniconda3/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180549130/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 819 layers, 164893160 parameters, 0 gradients, 225.7 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



In [77]:
# directory with yolo results
labels_dir = "runs/detect/exp13/labels"
images_dir = "../test_dataset_test/test/images"
# images_dir = "data_for_yolo/data/images/test"
# get filenames of marked images
detected_images = [str(p) for p in Path(images_dir).rglob("*.jpg")]
# create a confidence lookup
conf_lookup = dict(zip(range(5), [0.08, 0.065, 0.07, 0.05, 0.1]))
# conf_lookup = dict(zip(range(5), [0.1 for _ in range(5)]))
# populate submission
report_data = []
for image in detected_images:
    # open the predicted labels
    try:
        with open(f"{labels_dir}/{image.split('/')[-1].replace('.jpg', '.txt')}", 'r') as f:
            labels = [line.split(' ') for line in f.read().split('\n') if line not in ['\n', '']]
            # work with detected instance separately
            # print(labels)
            for class_label, *xywh, conf in labels:
                # print(class_label, conf, [x, y, w, h])
                x, y, w, h = [float(x) for x in xywh]
                xmin = x - w / 2
                xmax = x + w / 2
                ymin = y - h / 2
                ymax = y + h / 2
                conf = float(conf)
                class_label = int(class_label)
                # if class_label == 2 and conf > 0.01:
                #     report_data.append([image.split('/')[-1], class_label, conf, xmin, xmax, ymin, ymax])
                # elif class_label != 2 and conf > 0.1:
                #     report_data.append([image.split('/')[-1], class_label, conf, xmin, xmax, ymin, ymax])
                if conf > conf_lookup[class_label]:
                    report_data.append([image.split('/')[-1], class_label, conf, xmin, xmax, ymin, ymax])
    except:
        print(image)
# create dataframe out of all of it
df = pd.DataFrame(report_data, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])
# save dataframe
df.to_csv("../report_last.csv", sep=';', index=False)
# stats info
df.groupby("LabelName")["ImageID"].count()

LabelName
0    8138
1    1890
2     702
3    2310
4     118
Name: ImageID, dtype: int64

In [69]:
prev_report = pd.read_csv("../report_616.csv", sep=';')
prev_report.groupby("LabelName")["ImageID"].count()

LabelName
0    8610
1    2190
2     746
3    2863
4     122
Name: ImageID, dtype: int64